In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Note: using Google CoLab


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import numpy as np
from sklearn import metrics

In [3]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/m3gofriends/My-Machine-and-Deep-Learning-Template-main/main/TrainingSet/auto-mpg.csv?token=GHSAT0AAAAAABYGU6UEF7ZSFVWGBO6DN7E4YYVNYGA", 
    na_values=['NA', '?'])

In [4]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [5]:
# Remove target(Because the name of the car will hinder the training)
cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['acceleration'].values # regression

In [6]:
# Build the neural network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

Epoch 1/100
13/13 - 3s - loss: 40461.1211 - 3s/epoch - 215ms/step
Epoch 2/100
13/13 - 0s - loss: 8863.4688 - 30ms/epoch - 2ms/step
Epoch 3/100
13/13 - 0s - loss: 2020.9167 - 30ms/epoch - 2ms/step
Epoch 4/100
13/13 - 0s - loss: 479.9883 - 29ms/epoch - 2ms/step
Epoch 5/100
13/13 - 0s - loss: 186.1482 - 28ms/epoch - 2ms/step
Epoch 6/100
13/13 - 0s - loss: 98.5199 - 32ms/epoch - 2ms/step
Epoch 7/100
13/13 - 0s - loss: 76.4535 - 30ms/epoch - 2ms/step
Epoch 8/100
13/13 - 0s - loss: 68.6368 - 29ms/epoch - 2ms/step
Epoch 9/100
13/13 - 0s - loss: 66.4044 - 27ms/epoch - 2ms/step
Epoch 10/100
13/13 - 0s - loss: 63.7588 - 28ms/epoch - 2ms/step
Epoch 11/100
13/13 - 0s - loss: 61.7137 - 28ms/epoch - 2ms/step
Epoch 12/100
13/13 - 0s - loss: 61.3859 - 29ms/epoch - 2ms/step
Epoch 13/100
13/13 - 0s - loss: 59.0968 - 29ms/epoch - 2ms/step
Epoch 14/100
13/13 - 0s - loss: 55.6469 - 28ms/epoch - 2ms/step
Epoch 15/100
13/13 - 0s - loss: 54.8007 - 28ms/epoch - 2ms/step
Epoch 16/100
13/13 - 0s - loss: 52.7780 

In [7]:
pred = model.predict(x)
print(pred[0:10])

[[13.916545 ]
 [12.535197 ]
 [13.446116 ]
 [12.703319 ]
 [13.626811 ]
 [12.454967 ]
 [11.873485 ]
 [12.0468855]
 [11.457438 ]
 [12.662395 ]]


In [8]:
# Measure MSE error.  MSE is common for regression.
score = metrics.mean_squared_error(pred,y)
print(f"Final score (MSE): {score}")

Final score (MSE): 8.637130543165359


In [9]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Final score (RMSE): {score}")

Final score (RMSE): 2.938899546286902


In [10]:
# Sample predictions
for i in range(10):
    print(f"{i+1}. Car name: {cars[i]}, Acceleration: {y[i]}, predicted Acceleration: {pred[i]}.")

1. Car name: chevrolet chevelle malibu, Acceleration: 12.0, predicted Acceleration: [13.916545].
2. Car name: buick skylark 320, Acceleration: 11.5, predicted Acceleration: [12.535197].
3. Car name: plymouth satellite, Acceleration: 11.0, predicted Acceleration: [13.446116].
4. Car name: amc rebel sst, Acceleration: 12.0, predicted Acceleration: [12.703319].
5. Car name: ford torino, Acceleration: 10.5, predicted Acceleration: [13.626811].
6. Car name: ford galaxie 500, Acceleration: 10.0, predicted Acceleration: [12.454967].
7. Car name: chevrolet impala, Acceleration: 9.0, predicted Acceleration: [11.873485].
8. Car name: plymouth fury iii, Acceleration: 8.5, predicted Acceleration: [12.0468855].
9. Car name: pontiac catalina, Acceleration: 10.0, predicted Acceleration: [11.457438].
10. Car name: amc ambassador dpl, Acceleration: 8.5, predicted Acceleration: [12.662395].
